In [ ]:
# from utils import preprocess, train_test_split_data
from importlib import reload
import os
import utils
utils = reload(utils)

## Task 1: Data preparation

In [ ]:
# import nltk, nltk.data

# nltk.download('punkt')

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# with open('data/alice_in_wonderland.txt') as f:
#     text = f.read()
#     print('\n-----\n'.join(tokenizer.tokenize(text)))

In [ ]:
# sentencepiece package look for sentence hence linebreak needs to be preserved

with open('data/alice_in_wonderland.txt') as f:
    text = f.read()
    prepro_text = utils.preprocess(text) # preprocessing text
    train, test = utils.train_test_split_data(prepro_text, test_size=0.2) # split the data


In [ ]:
# saving the splitted corpus
with open('eng_text/train_eng.txt', 'w') as f:
    # f.write(str(train))
    f.write('\n'.join(train))

with open('eng_text/test_eng.txt', 'w') as f:
    # f.write(str(test)) 
    f.write('\n'.join(test))

### Comments/ideas
* split is not randomized
* more steps like lemmatization/stemming can be added in prepro
* there is not much preprocessing needs to be done
* may be some special characters/ punctuation can be removed

## Task 2: Subword Segmentation

### 1. Granularity: characters

In [ ]:
# training model

## CHANGE PATH TO SHIFT .model & .vocab in models/sentencepiece

# total 72 different types of characters
# coverage changes for non-english

n_ch = 72

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s1_train \
  --vocab_size=$n_ch \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s1_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s1_train.txt'

In [ ]:
# training model - test set

## CHANGE PATH TO SHIFT .model & .vocab in models/sentencepiece

# total 72 different types of characters
# coverage changes for non-english


!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s1_test \
  --vocab_size=$n_ch \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original) - test set

!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s1_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s1_test.txt'

### 2. Granularity: subword units (smaller vocabulary)

In [ ]:
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

small_vocab = 450

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s2_train \
  --vocab_size=$small_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original) -train
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s2_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s2_train.txt'

In [ ]:
# training model - test set

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s2_test \
  --vocab_size=$small_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s2_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s2_test.txt'

### 3. Granularity: subword units (larger vocabulary)

In [ ]:
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 1500-3000 for best performance
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

large_vocab = 2000

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s3_train \
  --vocab_size=$large_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s3_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s3_train.txt'

In [ ]:
# training model - test set

# coverage changes for non-english
# fine-tune vocab_size in range of 1500-3000 for best performance
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s3_test \
  --vocab_size=$large_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

In [ ]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s3_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s3_test.txt'

## Observation
* Character - Almost every single character is segmented
* Subword unit (smaller vacob) - The length of segmented subword is longer and many words are also considered as subwords
* Subword unit (larger vocab) - Here, subwords are longer. Many words are themself segmented into single subwords. It can also be seen consistently that orignally longer words are broken into two or more segments.  


## Task 3: LM Training

### 1. Baseline (en_s1 - character level)

In [35]:
# Training baseline LM (en_s1)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size

!cd models/rnnlm \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s1_train.txt' \
    -valid '../../eng_text/en_s1_test.txt' \
    -rnnlm baseline_en_s1 \
      -hidden 40 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 3 \
      -class $n_ch

debug mode: 2
train file: ../../eng_text/en_s1_train.txt
valid file: ../../eng_text/en_s1_test.txt
class size: 72
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: baseline_en_s1
Starting training using file ../../eng_text/en_s1_train.txt
Vocab size: 70
Words in train file: 115760
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 3.1281    Words/sec: 118621.0   VALID entropy: 3.4054
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 2.6953    Words/sec: 117046.6   VALID entropy: 3.3093
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 2.5930    Words/sec: 117524.9   VALID entropy: 3.3068
Iter:   3	Alpha: 0.050000	   TRAIN entropy: 2.4821    Words/sec: 113096.9   VALID entropy: 3.2160
Iter:   4	Alpha: 0.025000	   TRAIN entropy: 2.4283    Words/sec: 107167.5   VALID entropy: 3.1463
Iter:   5	Alpha: 0.012500	   TRAIN entropy: 2.4027    Words/sec: 119945.3   VALID entropy: 3.0978
Iter:   6	Alpha: 0.006250	   TRAIN entropy: 2.3894    Words/sec: 119817.9   VALID entropy: 3.0674
Iter:   7	Alpha: 0.003125	 

### 2. Baseline (smaller vocab - en_s2)
 

In [36]:
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s2_train.txt' \
    -valid '../../eng_text/en_s2_test.txt' \
    -rnnlm baseline_en_s2 \
      -hidden 40 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 3 \
      -class $small_vocab

debug mode: 2
train file: ../../eng_text/en_s2_train.txt
valid file: ../../eng_text/en_s2_test.txt
class size: 450
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: baseline_en_s2
Starting training using file ../../eng_text/en_s2_train.txt
Vocab size: 442
Words in train file: 46742
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 7.5093    Words/sec: 30811.5   VALID entropy: 7.1103
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 6.6438    Words/sec: 30687.7   VALID entropy: 6.5573
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 5.9915    Words/sec: 30949.6   VALID entropy: 6.3010
Iter:   3	Alpha: 0.100000	   TRAIN entropy: 5.5659    Words/sec: 30304.5   VALID entropy: 6.1937
Iter:   4	Alpha: 0.100000	   TRAIN entropy: 5.2843    Words/sec: 28543.6   VALID entropy: 6.1650
Iter:   5	Alpha: 0.100000	   TRAIN entropy: 5.0921    Words/sec: 30773.7   VALID entropy: 6.1694
Iter:   6	Alpha: 0.050000	   TRAIN entropy: 5.0307    Words/sec: 30378.9   VALID entropy: 6.0552
Iter:   7	Alpha: 0.025000	   TRAI

### 3. Baseline (larger vocab - en_s3)

In [39]:
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s3_train.txt' \
    -valid '../../eng_text/en_s3_test.txt' \
    -rnnlm baseline_en_s3 \
      -hidden 40 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 3 \
      -class $large_vocab

debug mode: 2
train file: ../../eng_text/en_s3_train.txt
valid file: ../../eng_text/en_s3_test.txt
class size: 2000
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: baseline_en_s3
Starting training using file ../../eng_text/en_s3_train.txt
Vocab size: 1829
Words in train file: 32645
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 8.2916    Words/sec: 6853.0   VALID entropy: 7.1652
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 7.5869    Words/sec: 6877.2   VALID entropy: 6.7712
Iter:   2	Alpha: 0.100000	   TRAIN entropy: 7.1646    Words/sec: 6753.5   VALID entropy: 6.6217
Iter:   3	Alpha: 0.100000	   TRAIN entropy: 6.8658    Words/sec: 6834.8   VALID entropy: 6.4476
Iter:   4	Alpha: 0.100000	   TRAIN entropy: 6.6328    Words/sec: 6851.3   VALID entropy: 6.3239
Iter:   5	Alpha: 0.100000	   TRAIN entropy: 6.4401    Words/sec: 6609.2   VALID entropy: 6.2968
Iter:   6	Alpha: 0.100000	   TRAIN entropy: 6.2748    Words/sec: 6763.2   VALID entropy: 6.2871
Iter:   7	Alpha: 0.050000	   TRAIN ent

In [ ]:
# Training baseline LM (en_s1)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size

!cd models/rnnlm \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s1_train.txt' \
    -valid '../../eng_text/en_s1_test.txt' \
    -rnnlm en_s1 \
      -hidden 40 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 3 \
      -class $n_ch

In [ ]:
# Training LM on subword units (smaller vocab - eng_s2)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size
# How to increase no. of iterations?

#incr hidden -> better pp
#incr bptt -> worse pp
# if class < vocabsize -> worse pp
#incr class -> better pp

## May be need to simplify the corpus (in preprocessing)

      # -bptt-block 1 \
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s2_train.txt' \
    -valid '../../eng_text/en_s2_test.txt' \
    -rnnlm en_s2 \
      -hidden 100 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 0 \
      -class $small_vocab

In [ ]:
# Training LM on subword units (larger vocab - eng_s3)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size
# How to increase no. of iterations?

#incr hidden -> better pp
#incr bptt -> worse pp
# if class < vocabsize -> worse pp
#incr class -> better pp

## May be need to simplify the corpus (in preprocessing)

      # -bptt-block 1 \
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s3_train.txt' \
    -valid '../../eng_text/en_s3_test.txt' \
    -rnnlm en_s3 \
      -hidden 120 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 0 \
      -class $large_vocab

## Task 4: Text Generation

In [ ]:
## 1. Character level granularity

!cd models/rnnlm/ \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm  en_s1\
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s1/${i}.txt"; \
       done

In [ ]:
## 2. subword units (smaller vocab - en_s2)

!cd models/rnnlm \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm en_s2 \
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s2/${i}.txt"; \
       done

In [ ]:
## 3. subword units (larger vocab - en_s3)

!cd models/rnnlm \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm en_s3 \
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s3/${i}.txt"; \
       done

In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s1_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s1/100.txt" \
  > "../../eng_text/gen_en_s1/decod_100.txt"

In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s2_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s2/100.txt" \
  > "../../eng_text/gen_en_s2/decod_100.txt"

In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s3_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s3/100.txt" \
  > "../../eng_text/gen_en_s3/decod_100.txt"